## Notebook_5 Tratamento de dados - Parte 1 
Preparação e transformaçãos 

In [1]:
import pandas as pd
import numpy as np
from datetime import date, datetime
pd.set_option('display.max_columns',None)

In [2]:
def transf_string_inteiro_soma(coluna2):
    coluna = []
    for i in coluna2:
        soma = 0
        if type(i) == list:
            for x in i:
                x = int(x)
                soma += x
            coluna.append(soma)    
        else: 
            coluna.append(i)
    return coluna

def transf_string_inteiro(coluna2):
    coluna = []
    for i in coluna2:
        lista = []
        if type(i) == list:
            for x in i:
                x = int(x)
                lista.append(x)    
            coluna.append(lista)    
        else: 
            coluna.append(i)
    return coluna

def transformar_em_data_chegada(dt):
    if len(dt) > 7:
        dia, mes, ano = int(dt[0:2]), int(dt[2:4]), int(dt[4:])
        data = date(ano, mes, dia)
    if len(dt) < 8:
        dia, mes, ano = int(dt[0]), int(dt[1:3]), int(dt[3:])
        data = date(ano, mes, dia)
    return data

def transformar_em_data_chegada_merge(dt):
    dia, mes, ano = str(dt[0:2]), str(dt[3:5]), str(dt[6:10])
    data = dia + mes + ano
    return data

In [3]:
df_trat = pd.read_csv('df_principal_pnr_formapgto.csv',sep=';')
df_trat.columns = df_trat.columns.str.lower().str.replace(' ', '_').str.replace('código_do_vu', 'cod_vu')
df_trat = df_trat.drop(columns=['unnamed:_0'])
df_trat = df_trat.drop_duplicates()

df_trat.shape

(26116, 50)

------------------------
Código do VU

In [4]:
df_trat['cod_vu'] = df_trat['cod_vu'].astype(str)

Número do voo

In [5]:
df_trat['numero_do_voo'] = df_trat['numero_do_voo'].astype(int).astype(str)
df_trat['numero_do_voo'] = 'Z' + df_trat['numero_do_voo'].astype('str') # para ficar categorico

Tratando data de nascimento 

In [6]:
# df_trat.loc[df_trat['datanascimento_vu'].isna().sum()]
df_trat['datanascimento_vu'] = df_trat['datanascimento_vu'].fillna('29/03/1988')
df_trat['datanascimento_vu'].isna().sum()

0

In [7]:
def transformar_em_data_nascimento(dt):
    dia, mes, ano = int(dt[0:2]),int(dt[3:5]),int(dt[6:10])
    data = date(ano, mes, dia)
    return data

data_lista, data_lista2 = list(df_trat['datanascimento_vu']), []
for i in data_lista:
    nova_dt = transformar_em_data_nascimento(i)
    data_lista2.append(transformar_em_data_nascimento(i))
df_trat['datanascimento_vu'] = data_lista2


Tratando a data de chegada

In [8]:
data_lista, data_lista2 = list(df_trat['data_da_chegada_do_voo2'].astype(str)), []
for i in data_lista:
    data_lista2.append(transformar_em_data_chegada(i))

df_trat['data_da_chegada_do_voo'] = data_lista2

-----------------------------
FISCALIZAÇÃO
###### Verifica quantas fiscalizações anteriores no período foram realizadas antes do registro da fiscalização em questão

In [9]:
# Para comparação com os fiscalizados
df_trat['viagens_periodo_do_vu_trat'] = 1
df_trat['viagens_periodo_do_vu_trat'] = df_trat.groupby('cod_vu')['viagens_periodo_do_vu_trat'].transform(np.sum)
df_trat = df_trat.sort_values('data_da_chegada_do_voo')

In [10]:
df_trat = df_trat.sort_values('data_da_chegada_do_voo',ascending=False)
dic_contagem, lista_qtd = {} , []

for i in df_trat['cod_vu']:
    if i in dic_contagem:
        dic_contagem[i] += 1
        lista_qtd.append(dic_contagem[i])  
    else:
        lista_qtd.append(1)
        dic_contagem[i] = 1

df_trat['qtd_fisc_descendente_trat'] = lista_qtd
df_trat = df_trat.sort_values('data_da_chegada_do_voo')
df_trat['fiscalizacoes_qt_vu_trat'] = df_trat['fiscalizacoes_qt_vu'] - df_trat['qtd_fisc_descendente_trat']
df_trat['fiscalizacoes_qt_vu_trat'] = df_trat['fiscalizacoes_qt_vu_trat'].apply(lambda x: 0 if x<0 else x).fillna(13)
# df_trat[['cod_vu','qtd_fisc_descendente_trat','viagens_periodo_do_vu_trat',
#         'fiscalizacoes_qt_vu','fiscalizacoes_qt_vu_trat'
#         ]].loc[df_trat['viagens_periodo_do_vu_trat']>1].sort_values('cod_vu').head(4)

-----------------
VIAGENS - Chegada - API
###### Vericando o número de viagens na data de cada fiscalização a partir de todos os arquivos API do período

In [11]:
#arquivos do API - esse API é todos os voos, de entrada e de saída do país. 
api1 = pd.read_csv('VOOS jan mar.csv',sep=';', encoding = 'latin1')
api2 = pd.read_csv('VOOS abr jun.csv',sep=';', encoding = 'latin1')
df_api = pd.concat([api1,api2])
df_api.shape

(1500586, 16)

Criando um código para o merge 

In [12]:
lista_data, lista_data_merge = list(df_api['Data da Chegada do Voo']), []
for i in lista_data:
    lista_data_merge.append(transformar_em_data_chegada_merge(i)) # transformando a data da chegada
df_api['Data da Chegada do Voo merge'] = lista_data_merge
df_api['vu chegada'] = df_api['Numero do Viajante Unico'].astype(str) + df_api['Data da Chegada do Voo merge']

In [14]:
# retirando repetidos (parte 1) por meio de um dicionário. Aqui, verifica se há repeditos para depois retirar esses repetidos com LOC
dic_contagem, lista_qtd  = {}, []
for i in df_api['vu chegada']:
    if i in dic_contagem:
        lista_qtd.append(dic_contagem[i])
        dic_contagem[i] += 1
    else:
        lista_qtd.append(0)
        dic_contagem[i] = 1

df_api['num_viagens_api'] = lista_qtd
# df_api[['Numero do Viajante Unico','num_viagens_api']].loc[df_api['num_viagens_api']>1].sort_values('Numero do Viajante Unico').head(2)

#Retirando repeditos (parte 2)
df_api = df_api.loc[df_api['num_viagens_api'] == 0] 

# somando o número de viagens para comparação
df_api['num_viagens_api'] = 1
df_api['num_viagens_api'] = df_api.groupby('Numero do Viajante Unico')['num_viagens_api'].transform(np.sum)
# df_api.loc[df_api['num_viagens_api']>1].sort_values('Numero do Viajante Unico').head(4)

In [15]:
# contado viagens anteriores à data da viagem atual.
df_api, dic_contagem, lista_qtd = df_api.sort_values('Data da Chegada do Voo',ascending=False), {}, []
for i in df_api['Numero do Viajante Unico']:
    if i in dic_contagem:
        dic_contagem[i] += 1
        lista_qtd.append(dic_contagem[i])
    else:
        lista_qtd.append(1)
        dic_contagem[i] = 1

df_api['num_viagens_api_subtrair'] = lista_qtd

df_api = df_api[['vu chegada','num_viagens_api_subtrair']]

-------------------------------------------------
Merge com o df_trat - número de viagens

In [16]:
lista_data = list(df_trat['data_da_chegada_do_voo'])
lista_dia, lista_mes, lista_ano = [], [], []
for i in lista_data:
    lista_dia.append(str(i.strftime('%d'))), lista_mes.append(str(i.strftime('%m'))), lista_ano.append(str(i.strftime('%Y')))

df_trat['data_merge_dia'], df_trat['data_merge_mes'], df_trat['data_merge_ano'] = lista_dia, lista_mes,lista_ano
df_trat['data_merge'] = df_trat['cod_vu'].astype(str) + df_trat['data_merge_dia'] + df_trat['data_merge_mes'] + df_trat['data_merge_ano']

In [17]:
df_trat.shape,df_api.shape

((26116, 58), (1462240, 2))

In [18]:
df_trat = pd.merge(df_trat,df_api, left_on='data_merge',right_on='vu chegada', how='left')
df_trat.shape

(26116, 60)

In [19]:
# Trando o número de viagens.
df_trat['qtviagens_vu_trat'] = df_trat['qtviagens_vu'] - df_trat['num_viagens_api_subtrair']
df_trat['qtviagens_vu_trat'] = df_trat['qtviagens_vu_trat'].fillna(219) #apenas um passageiros que foi consultado
df_trat[['qtviagens_vu','num_viagens_api_subtrair', 'qtviagens_vu_trat']].sample(2)

,qtviagens_vu,num_viagens_api_subtrair,qtviagens_vu_trat
108,2.0,2.0,0.0
6318,2.0,1.0,1.0


-----------------
OCORRÊNCIAS: calculando o número de ocorrência das informações do VU para quem deu resultado. 

In [20]:
df_trat = df_trat.sort_values('data_da_chegada_do_voo',ascending=False)
ls_vu, ls_resultado = list(df_trat['cod_vu']), list(df_trat['resultado_reserva']) 
dic_contagem, ls_hist_resultado = {}, []

index = 0
for vu in ls_vu: 
    if vu in dic_contagem:
        if ls_resultado[index] == 0:
            ls_hist_resultado.append(dic_contagem[vu])
        if ls_resultado[index] > 0:
            dic_contagem[vu] += 1
            ls_hist_resultado.append(dic_contagem[vu])
    if vu not in dic_contagem:
        if ls_resultado[index] == 0:
            dic_contagem[vu] = 0
            ls_hist_resultado.append(0)
        else:
            dic_contagem[vu] = 1
            ls_hist_resultado.append(1)
    index += 1
            
df_trat['declaracoes_qt_vu_trat'] = ls_hist_resultado
df_trat['declaracoes_qt_vu_trat'] = df_trat['declaracoes_qt_vu'] - df_trat['declaracoes_qt_vu_trat']
df_trat['declaracoes_qt_vu_trat'] = np.where(df_trat['declaracoes_qt_vu_trat'] <0, 0, df_trat['declaracoes_qt_vu_trat'])

df_trat['declaracoes_qt_vu_trat'] = df_trat['declaracoes_qt_vu_trat'].fillna(0) #apenas um passageiros que foi consultado

df_trat[['declaracoes_qt_vu','resultado_reserva','declaracoes_qt_vu_trat']].sort_values('resultado_reserva', ascending=False).head(4)


,declaracoes_qt_vu,resultado_reserva,declaracoes_qt_vu_trat
12648,4.0,1,3.0
2047,1.0,1,0.0
20637,1.0,1,0.0
20636,2.0,1,1.0


Documento - validade - ano - nan

In [21]:
df_trat['documento_validade_vu'] = pd.to_datetime(df_trat['documento_validade_vu'],format='%d/%m/%Y',errors='ignore')
df_trat['doc_ano_validade_trat'] = df_trat['documento_validade_vu'].dt.year
df_trat['doc_ano_validade_trat_2'] = df_trat['doc_ano_validade_trat'].loc[df_trat['doc_ano_validade_trat'].notna()]
df_trat['doc_ano_validade_trat'] = df_trat['documento_validade_vu'].fillna(df_trat['doc_ano_validade_trat_2'].median())
df_trat = df_trat.drop(columns=['doc_ano_validade_trat_2'])

Nacionalidade nan
###### identifiquei que os viajantes com nacionalidade faltante eram Brasileiros. 2

In [22]:
# df_trat.loc[df_trat['nacionalidade'].isna()]
df_trat['nacionalidade'] = df_trat['nacionalidade'].fillna('BRA') 

Dia da semana data de chegada


In [23]:
lista_data, lista_dia_semana = list(df_trat['data_da_chegada_do_voo']), []
for i in lista_data:
    lista_dia_semana.append(i.isoweekday())
df_trat['dia_semana_chegada_trat'] = lista_dia_semana

CPF e CPF do responsável

In [24]:
df_trat['cpf_trat'] = np.where(df_trat['cpf_vu'].notna(), True,df_trat['cpf_vu'])
df_trat['cpfresponsavel_vu'] = np.where(df_trat['cpfresponsavel_vu'].notna(), True,df_trat['cpfresponsavel_vu'])

Região Fiscal do CPF

In [25]:
df_trat['cpf_regiao_fiscal_trat'] = np.select([
                                    (df_trat['cpf_trat']==True)], 
                                    [(df_trat['cpf_vu'].str.slice(start=10,stop=11))], 
                                    default=df_trat['cpf_vu'])
                                    

-----------------
CNPJ

Quantidade de CNPJ's e estabelecimentos

In [26]:
df_trat['cnpj_qtd_trat'] = df_trat['cnpj_cod_vu'].str.replace('[','').str.replace(']','').str.replace(' ','').str.split(',')
df_trat['cnpj_qtd_trat'] = df_trat['cnpj_qtd_trat'].apply(lambda x: len(x) if type(x) == list else x)

df_trat['cnpj_nrestabelecimento_trat'] = df_trat['cnpj_nrestabelecimento_vu'].str.replace('[','').str.replace(']','').str.replace(' ','').str.split(',')
df_trat['cnpj_nrestabelecimento_trat'] = transf_string_inteiro_soma(df_trat['cnpj_nrestabelecimento_trat'])


C:\Users\066176~1.INT\AppData\Local\Temp/ipykernel_920/1039586289.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_trat['cnpj_qtd_trat'] = df_trat['cnpj_cod_vu'].str.replace('[','').str.replace(']','').str.replace(' ','').str.split(',')
C:\Users\066176~1.INT\AppData\Local\Temp/ipykernel_920/1039586289.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_trat['cnpj_nrestabelecimento_trat'] = df_trat['cnpj_nrestabelecimento_vu'].str.replace('[','').str.replace(']','').str.replace(' ','').str.split(',')


Verificando a situação dos CPNJs

In [27]:
df_trat['situacao_cnpj_ativa_trat'] = df_trat['cnpj_situacao_vu'].str.contains('ATIVA')
df_trat['situacao_cnpj_ativa_trat'] = np.where(df_trat['situacao_cnpj_ativa_trat'] == True, 1, 0)
df_trat['situacao_cnpj_ativa_trat'] = np.select([(df_trat['cnpj_situacao_vu'].isna())], [(df_trat['cnpj_situacao_vu'])] , default=df_trat['situacao_cnpj_ativa_trat'])

df_trat['situacao_cnpj_baixada_trat'] = df_trat['cnpj_situacao_vu'].str.contains('BAIXADA')
df_trat['situacao_cnpj_baixada_trat'] = np.where(df_trat['situacao_cnpj_baixada_trat'] == True, 1, 0)
df_trat['situacao_cnpj_baixada_trat'] = np.select([(df_trat['cnpj_situacao_vu'].isna())], [(df_trat['cnpj_situacao_vu'])] , default=df_trat['situacao_cnpj_baixada_trat'])

df_trat['situacao_cnpj_inapta_trat'] = df_trat['cnpj_situacao_vu'].str.contains('INAPTA')
df_trat['situacao_cnpj_inapta_trat'] = np.where(df_trat['situacao_cnpj_inapta_trat'] == True, 1, 0)
df_trat['situacao_cnpj_inapta_trat'] = np.select([(df_trat['cnpj_situacao_vu'].isna())], [(df_trat['cnpj_situacao_vu'])], default=df_trat['situacao_cnpj_inapta_trat'])

df_trat['situacao_cnpj_nula_trat'] = df_trat['cnpj_situacao_vu'].str.contains('NULA')
df_trat['situacao_cnpj_nula_trat'] = np.where(df_trat['situacao_cnpj_nula_trat'] == True, 1, 0)
df_trat['situacao_cnpj_nula_trat'] = np.select([(df_trat['cnpj_situacao_vu'].isna())], [(df_trat['cnpj_situacao_vu'])], default=df_trat['situacao_cnpj_nula_trat'])

df_trat['situacao_cnpj_suspensa_trat'] = df_trat['cnpj_situacao_vu'].str.contains('SUSPENSA')
df_trat['situacao_cnpj_suspensa_trat'] = np.where(df_trat['situacao_cnpj_suspensa_trat'] == True, 1, 0)
df_trat['situacao_cnpj_suspensa_trat'] = np.select([(df_trat['cnpj_situacao_vu'].isna())], [(df_trat['cnpj_situacao_vu'])], default=df_trat['situacao_cnpj_suspensa_trat'])


---------------------
Gênero

In [28]:
#df_trat['genero_vu'].isna().sum()
#df_trat.loc[df_trat['genero_vu'].isna()]
df_trat['genero_vu'] = df_trat['genero_vu'].fillna('M')
df_trat['genero_vu'] = df_trat['genero_vu'].str.replace('U','F')

País_emissor_do_documento_principal


In [29]:
print(df_trat['país_emissor_do_documento_principal'].isna().sum())
# df_trat.loc[df_trat['país_emissor_do_documento_principal'].isna()]
df_trat.drop(df_trat[df_trat['país_emissor_do_documento_principal'].isna()].index, inplace=True)


16


EMISSOR DOCUMENTO

In [30]:
#df_trat['documento_emissor_vu'].isna().sum()
#df_trat.loc[df_trat['documento_emissor_vu'].isna()]
# df_trat.drop(df_trat[df_trat['cod_vu']== '20200107191840032'].index, inplace=True)
df_trat['documento_emissor_vu'] = df_trat['documento_emissor_vu'].fillna('CONGO, REPUBLICA DEMOCRATICA DO')
df_trat['documento_emissor_vu'].isna().sum()


0

Qtdd de documentos utilizados

In [31]:
print(df_trat['doc_numdocumento_hist_vu'].isna().sum())
df_trat['doc_qtd_trat'] = df_trat['doc_numdocumento_hist_vu'].str.replace('[','').str.replace(']','').str.split(',')
df_trat['doc_qtd_trat'] = df_trat['doc_qtd_trat'].apply(lambda x: len(x) if type(x) == list else x)
df_trat.drop(df_trat[df_trat['doc_qtd_trat'].isna()].index, inplace=True)

314


C:\Users\066176~1.INT\AppData\Local\Temp/ipykernel_920/3085018881.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_trat['doc_qtd_trat'] = df_trat['doc_numdocumento_hist_vu'].str.replace('[','').str.replace(']','').str.split(',')


Idade na data da fiscalização 

In [32]:
df_trat['idade_trat'] = (df_trat['data_da_chegada_do_voo'] - df_trat['datanascimento_vu']).astype('timedelta64[Y]')


MONITORADO e INFORMAÇÃO DE INTERESSE

In [33]:
# O nan foram veritificados manualmente
print(df_trat['monitoramentos_qt_vu'].isna().sum())
# df_trat.loc[df_trat['monitoramentos_qt_vu'].isna()]
df_trat['monitoramentos_qt_vu'] = df_trat['monitoramentos_qt_vu'].fillna(0)

1


In [34]:
# Os nan foram veritificados manualmente
print(df_trat['infosinteresse_qt_vu'].isna().sum())
# df_trat.loc[df_trat['infosinteresse_qt_vu'].isna()]
df_trat['infosinteresse_qt_vu'] = df_trat['infosinteresse_qt_vu'].fillna(0)

1


DOCUMENTOS - N. DE PAÍSES EMISSORES

In [35]:
df_trat['doc_qtd__paises_trat'] = df_trat['doc_paisemissor_hist_vu'].str.replace('[','').str.replace(']','').str.replace(' ','').str.split(',')
df_trat['doc_qtd__paises_trat'] = df_trat['doc_qtd__paises_trat'].apply(lambda x: len(set(x)) if type(x) == list else x)
df_trat[['doc_paisemissor_hist_vu','doc_qtd__paises_trat']].head(5)


C:\Users\066176~1.INT\AppData\Local\Temp/ipykernel_920/3374697007.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_trat['doc_qtd__paises_trat'] = df_trat['doc_paisemissor_hist_vu'].str.replace('[','').str.replace(']','').str.replace(' ','').str.split(',')


,doc_paisemissor_hist_vu,doc_qtd__paises_trat
26115,"[ESTADOS UNIDOS, BRASIL, ESTADOS UNIDOS, BRASIL]",2
26095,[ARGENTINA],1
26093,[BRASIL],1
26092,"[ESTADOS UNIDOS, BRASIL, BRASIL]",2
26091,"[ESTADOS UNIDOS, BRASIL, ESTADOS UNIDOS, BRASIL]",2


IATA DE ORIGEM - quantidade de vezes que o passageiro já originou de um determinado aeroporto (iata)

In [36]:
df_trat['origem_ls_trat'] = df_trat['aeroportoorigemviajantehist_vu'].str.replace('[','').str.replace(']','').str.replace(' ','').str.split(',')
df_trat['origem_ls_trat'] = df_trat['origem_ls_trat'].fillna('[]') 
df_trat['qtviagens_vu_trat'] = df_trat['qtviagens_vu_trat'].astype(int) 

ls_iatas_origem = list(df_trat['origem_ls_trat']) 
ls_qtviagens = list(df_trat['qtviagens_vu_trat'])
ls_iatas_fisc = []
index = 0 
for i in ls_iatas_origem:
    iatas_fiscalizacao = i[-(ls_qtviagens[index]):]
    ls_iatas_fisc.append(iatas_fiscalizacao)
    index +=1

df_trat['origem_ls_trat_2'] = ls_iatas_fisc

lista_iatas_origem = list(df_trat['origem_ls_trat_2']) 
lista_iata_origem = list(df_trat['codigo_iata_do_local_de_embarque_do_passageiro'])
lista_iata_origem_qtd = []
index = 0
for i in lista_iatas_origem:
    j = lista_iata_origem[index]
    lista_iata_origem_qtd.append(i.count(j))
    index +=1 
df_trat['qtd_iata_origem_trat'] = lista_iata_origem_qtd
df_trat[['origem_ls_trat','qtviagens_vu_trat','origem_ls_trat_2','codigo_iata_do_local_de_embarque_do_passageiro','qtd_iata_origem_trat']].sample(2)


C:\Users\066176~1.INT\AppData\Local\Temp/ipykernel_920/3772846722.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_trat['origem_ls_trat'] = df_trat['aeroportoorigemviajantehist_vu'].str.replace('[','').str.replace(']','').str.replace(' ','').str.split(',')


,origem_ls_trat,qtviagens_vu_trat,origem_ls_trat_2,codigo_iata_do_local_de_embarque_do_passageiro,qtd_iata_origem_trat
16146,"[LIM, GRU, LIM, GRU, LIM, CWB, LIM, CWB, LIM, ...",15,"[GRU, LIM, GRU, LIM, CWB, LIM, CWB, LIM, GRU, ...",LIM,7
10331,[EZE],0,[EZE],EZE,1


QTD de países de origem

In [37]:
df_trat['qtd_paises_visitados_trat'] = df_trat['origem_qt_vu'].str.replace('[','').str.replace(']','').str.split(',') 
df_trat['qtd_paises_visitados_trat'] = df_trat['qtd_paises_visitados_trat'].apply(lambda x: len(x) if type(x) == list else x)


C:\Users\066176~1.INT\AppData\Local\Temp/ipykernel_920/2112734164.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_trat['qtd_paises_visitados_trat'] = df_trat['origem_qt_vu'].str.replace('[','').str.replace(']','').str.split(',')


 QTD países de destino

In [38]:
df_trat['qtd_paises_destino_trat'] = df_trat['destino_qt_vu'].str.replace('[','').str.replace(']','').str.split(',') 
df_trat['qtd_paises_destino_trat'] = df_trat['qtd_paises_destino_trat'].apply(lambda x: len(x) if type(x) == list else x)


C:\Users\066176~1.INT\AppData\Local\Temp/ipykernel_920/2005302288.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_trat['qtd_paises_destino_trat'] = df_trat['destino_qt_vu'].str.replace('[','').str.replace(']','').str.split(',')


Número de cia utilizadas pelo viajante

In [39]:
# df_trat.loc[df_trat['ciaaereahist_vu'].isna()]
df_trat['ciaaereahist_vu'] = df_trat['ciaaereahist_vu'].fillna('[UA, CM, AD]')
# df_trat.loc[df_trat['ciaaereahist_vu'].isna()]

df_trat['qtd_ciaaerea_trat'] = df_trat['ciaaereahist_vu'].str.replace('[','').str.replace(']','').str.replace(' ','').str.split(',')
lista_1,lista_2 = list(df_trat['qtd_ciaaerea_trat']),[]
for i in lista_1:
    lista_3 = i
    lista_4 = []
    for j in lista_3:
        lista_4.append(j.strip(' '))
    lista_2.append(len(set(lista_4)))

df_trat['qtd_ciaaerea_trat'] = lista_2
df_trat[['ciaaereahist_vu','qtd_ciaaerea_trat']].sample(4)

C:\Users\066176~1.INT\AppData\Local\Temp/ipykernel_920/1231432413.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_trat['qtd_ciaaerea_trat'] = df_trat['ciaaereahist_vu'].str.replace('[','').str.replace(']','').str.replace(' ','').str.split(',')


,ciaaereahist_vu,qtd_ciaaerea_trat
1452,"[EK, LH, AV, EK, AZ, EK, EK, EK, KL, EK, EK, E...",7
2197,"[AF, JJ, JJ, JJ, JJ, JJ, JJ, AZ, AZ, DL, DL, A...",8
14314,"[AA, AA, AA, AA, DL, DL, AA, AA, AA, AA, LA, G...",7
20690,"[AA, JJ, JJ, JJ, AA, AA, AA, AA, JJ, JJ, JJ, J...",9


Média de viajantes na mesma reserva
###### incluíndo todos os voos

In [40]:
df_trat['qtviajantesreservahist_vu2'] = df_trat['qtviajantesreservahist_vu'].str.replace('[','').str.replace(']','').str.replace(' ','').str.split(',')
df_trat['qtviajantesreservahist_vu2'].head(4)

C:\Users\066176~1.INT\AppData\Local\Temp/ipykernel_920/1171858832.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_trat['qtviajantesreservahist_vu2'] = df_trat['qtviajantesreservahist_vu'].str.replace('[','').str.replace(']','').str.replace(' ','').str.split(',')


26115    [1, 1, 2, 1, 1, 2, 1, 1, 1, 1]
26095             [1, 1, 1, 1, 1, 1, 1]
26093                            [2, 2]
26092             [2, 1, 1, 2, 2, 1, 1]
Name: qtviajantesreservahist_vu2, dtype: object

In [41]:
df_trat['num_viagens_api_subtrair'] = df_trat['num_viagens_api_subtrair'].fillna(0)


In [42]:
df_trat['qtviajantesreservahist_vu2'] = df_trat['qtviajantesreservahist_vu2'].fillna('[0]')

In [43]:
lista1, lista2, lista_res  = list(df_trat['qtviajantesreservahist_vu2']), list(df_trat['num_viagens_api_subtrair']), []
index = 0
for i in lista1:
    j = int(lista2[index])
    lista_res.append(i[j:])
    index +=1

lista_res2 = []
for i in lista_res:
    lista_res3 = []
    for j in i:
        try:
            lista_res3.append(int(j))
        except:
            lista_res3.append(j)
    try:
        mean = sum(lista_res3)/len(lista_res3)
        lista_res2.append(mean)
    except:
        lista_res2.append(-1)

df_trat['qtviajantesreservahist_vu_media'] = lista_res2
df_trat[['qtviajantesreservahist_vu2','num_viagens_api_subtrair','qtviajantesreservahist_vu_media']].sample(5)


,qtviajantesreservahist_vu2,num_viagens_api_subtrair,qtviajantesreservahist_vu_media
7644,"[1, 1, 1, 1]",1.0,1.000000
17930,"[3, 3, 3, 3, 1, 1, 1, 1, 3, 3, 1, 2, 1, 1, 1, ...",1.0,1.310345
5947,"[4, 4, 4, 4, 4, 4]",2.0,4.000000
464,"[1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2.0,1.023256
1575,"[1, 1]",1.0,1.000000


 RESERVA: Número de dias antes da chegada

In [44]:
def transformar_em_data_partida(dt):
    dia, mes, ano = int(dt[0:2]), int(dt[3:5]), int(dt[6:10])
    data = date(ano, mes, dia)
    return data

df_trat['data_reserva_pnr_trat'] = pd.to_datetime(df_trat['data_reserva_pnr'],format='%d/%m/%Y',errors='ignore')
df_trat['data_de_partida_do_voo_trat'] = df_trat['data_de_partida_do_voo'].apply(lambda x: x[0:10])

data_lista, data_lista2 = list(df_trat['data_de_partida_do_voo_trat']),[] # tratamento da data da partida
for i in data_lista:
    data_lista2.append(transformar_em_data_partida(i))
df_trat['data_de_partida_do_voo_trat'] = data_lista2

data_lista, data_lista2 = list(df_trat['data_reserva_pnr']), [] # tratamento da data da reserva
for i in data_lista:
    data_lista2.append(transformar_em_data_partida(i))
df_trat['data_reserva_pnr_trat'] = data_lista2

df_trat['reserva_dias_antes_partida_trat'] = df_trat['data_de_partida_do_voo_trat'] - df_trat['data_reserva_pnr_trat']
df_trat[['data_de_partida_do_voo','reserva_dias_antes_partida_trat','data_reserva_pnr']].sample(2)



,data_de_partida_do_voo,reserva_dias_antes_partida_trat,data_reserva_pnr
16985,23/04/2021 01:25:00,44 days,10/03/2021
12703,23/03/2021 07:20:00,14 days,09/03/2021


 Cidade de embarque

In [45]:
df_trat['cidade_do_embarque_do_viajante'] = df_trat['cidade_do_embarque_do_viajante'].fillna('Najran') #apenas um nan foi identificado

 Padronização da nacionalidade, pais_emissor do documento principal, pais de origem do voo, pais de embarque do viajante, pais emissor do documento

In [46]:
df_trat['nacionalidade'] = df_trat['nacionalidade'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.replace(
                            ' ','').str.lower().str.replace(',','').str.replace('armeniarepublicada','amermenia').str.replace('chinarepublicapopular','china').str.replace(
                            'congorepublicademocraticado','congo')
df_trat['nacionalidade']=df_trat['nacionalidade'].str.replace('caboverderepublicade','caboverde').str.replace('estoniarepublicada','estonia').str.replace(
                            'georgiarepublicada','georgia').str.replace('croacia(republicada)','croacia').str.replace('hungriarepublicada','hungria').str.replace(
                            'irarepublicaislamicado','ira').str.replace('cazaquistaorepublicado','cazaquistao').str.replace('coreia(dosul)republicada','coreiadosul').str.replace(
                            'russiafederacaoda','russia')

df_trat['país_emissor_do_documento_principal'] = df_trat['país_emissor_do_documento_principal'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.replace(
                            ' ','').str.lower().str.replace(',','').str.replace('armeniarepublicada','amermenia').str.replace('chinarepublicapopular','china').str.replace(
                            'congorepublicademocraticado','congo')
df_trat['país_emissor_do_documento_principal'] = df_trat['país_emissor_do_documento_principal'].str.replace('caboverderepublicade','caboverde').str.replace('estoniarepublicada','estonia').str.replace(
                            'georgiarepublicada','georgia').str.replace('croacia(republicada)','croacia').str.replace('hungriarepublicada','hungria').str.replace(
                            'irarepublicaislamicado','ira').str.replace('cazaquistaorepublicado','cazaquistao').str.replace('coreia(dosul)republicada','coreiadosul').str.replace(
                            'russiafederacaoda','russia')

df_trat['pais_de_origem_do_voo'] = df_trat['pais_de_origem_do_voo'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.replace(' ','').str.lower().str.replace(
                            ',','').str.replace('armeniarepublicada','amermenia').str.replace('chinarepublicapopular','china').str.replace(
                            'congorepublicademocraticado','congo')
df_trat['pais_de_origem_do_voo']= df_trat['pais_de_origem_do_voo'].str.replace('caboverderepublicade','caboverde').str.replace('estoniarepublicada','estonia').str.replace(
                            'georgiarepublicada','georgia').str.replace('croacia(republicada)','croacia').str.replace('hungriarepublicada','hungria').str.replace(
                            'irarepublicaislamicado','ira').str.replace('cazaquistaorepublicado','cazaquistao').str.replace('coreia(dosul)republicada','coreiadosul').str.replace(
                            'russiafederacaoda','russia')

df_trat['pais_de_embarque_do_viajante'] = df_trat['pais_de_embarque_do_viajante'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.replace(
                            ' ','').str.lower().str.replace(',','').str.replace('armeniarepublicada','amermenia').str.replace('chinarepublicapopular','china').str.replace(
                            'congorepublicademocraticado','congo')
df_trat['pais_de_embarque_do_viajante'] = df_trat['pais_de_embarque_do_viajante'].str.replace('caboverderepublicade','caboverde').str.replace('estoniarepublicada','estonia').str.replace(
                            'georgiarepublicada','georgia').str.replace('croacia(republicada)','croacia').str.replace('hungriarepublicada','hungria').str.replace(
                            'irarepublicaislamicado','ira').str.replace('cazaquistaorepublicado','cazaquistao').str.replace('coreia(dosul)republicada','coreiadosul').str.replace(
                            'russiafederacaoda','russia')

df_trat['documento_emissor_vu'] = df_trat['documento_emissor_vu'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.replace(
                            ' ','').str.lower().str.replace(',','').str.replace('armeniarepublicada','amermenia').str.replace('chinarepublicapopular','china').str.replace(
                            'congorepublicademocraticado','congo')
df_trat['documento_emissor_vu'] = df_trat['documento_emissor_vu'].str.replace('caboverderepublicade','caboverde').str.replace('estoniarepublicada','estonia').str.replace(
                            'georgiarepublicada','georgia').str.replace('croacia(republicada)','croacia').str.replace('hungriarepublicada','hungria').str.replace(
                            'irarepublicaislamicado','ira').str.replace('cazaquistaorepublicado','cazaquistao').str.replace('coreia(dosul)republicada','coreiadosul').str.replace(
                            'russiafederacaoda','russia')

C:\Users\066176~1.INT\AppData\Local\Temp/ipykernel_920/256207719.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_trat['nacionalidade']=df_trat['nacionalidade'].str.replace('caboverderepublicade','caboverde').str.replace('estoniarepublicada','estonia').str.replace(
C:\Users\066176~1.INT\AppData\Local\Temp/ipykernel_920/256207719.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df_trat['país_emissor_do_documento_principal'] = df_trat['país_emissor_do_documento_principal'].str.replace('caboverderepublicade','caboverde').str.replace('estoniarepublicada','estonia').str.replace(
C:\Users\066176~1.INT\AppData\Local\Temp/ipykernel_920/256207719.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
  df_trat['pais_de_origem_do_voo']= df_trat['pais_de_origem_do_voo'].str.replace('caboverderepublicade','caboverde').str.replace('estoniarepubl

In [47]:
cod_paises = pd.read_csv('paises_codigos.csv', sep=';')
cod_paises.head(4)

,cod,nacionalidade_trat
0,ago,angola
1,dom,dominicailha
2,alb,albania
3,are,emiradosarabesunidos


In [48]:
df_trat = pd.merge(df_trat,cod_paises,left_on='nacionalidade',right_on='cod', how='left')
df_trat['nacionalidade_trat'] = df_trat['nacionalidade_trat'].fillna('marrocos')
df_trat = df_trat.drop(columns=['nacionalidade','cod'])
df_trat.head(4)

,aeroporto_de_origem_do_voo,cod_vu,país_emissor_do_documento_principal,codigo_iata_do_destino_do_voo,data_de_partida_do_voo,pais_de_origem_do_voo,numero_do_voo,companhia_aerea,codigo_iata_do_local_de_embarque_do_passageiro,cidade_do_embarque_do_viajante,pais_de_embarque_do_viajante,codigo_iata_do_local_de_desembarque_do_viajante,quantidade_de_bagagens,data_da_chegada_do_voo2,qtd_pax_reserva,resultado_reserva,cpf_vu,cpfresponsavel_vu,nacionalidade_vu,genero_vu,documento_emissor_vu,documento_validade_vu,datanascimento_vu,qtviagens_vu,monitoramentos_qt_vu,fiscalizacoes_qt_vu,infosinteresse_qt_vu,declaracoes_qt_vu,dirpf_tipocontribuinte_vu,dirpf_totalrendimentotributavel_vu,dirpf_totalrendimentonaotributavel_vu,dirpf_totalbensdeclarados_vu,cnpj_cod_vu,cnpj_nrestabelecimento_vu,cnpj_situacao_vu,cnpj_atividadeeconomicapreferencial_vu,doc_numdocumento_hist_vu,doc_paisemissor_hist_vu,origem_lista_vu,origem_qt_vu,destino_lista_vu,destino_qt_vu,ciaaereahist_vu,numerovoohist_vu,qtviajantesreservahist_vu,aeroportoorigemviajantehist_vu,aeroportodestinoviajantehist_vu,data_reserva_pnr,forma_pgto_pnr,data_da_chegada_do_voo,viagens_periodo_do_vu_trat,qtd_fisc_descendente_trat,fiscalizacoes_qt_vu_trat,data_merge_dia,data_merge_mes,data_merge_ano,data_merge,vu chegada,num_viagens_api_subtrair,qtviagens_vu_trat,declaracoes_qt_vu_trat,doc_ano_validade_trat,dia_semana_chegada_trat,cpf_trat,cpf_regiao_fiscal_trat,cnpj_qtd_trat,cnpj_nrestabelecimento_trat,situacao_cnpj_ativa_trat,situacao_cnpj_baixada_trat,situacao_cnpj_inapta_trat,situacao_cnpj_nula_trat,situacao_cnpj_suspensa_trat,doc_qtd_trat,idade_trat,doc_qtd__paises_trat,origem_ls_trat,origem_ls_trat_2,qtd_iata_origem_trat,qtd_paises_visitados_trat,qtd_paises_destino_trat,qtd_ciaaerea_trat,qtviajantesreservahist_vu2,qtviajantesreservahist_vu_media,data_reserva_pnr_trat,data_de_partida_do_voo_trat,reserva_dias_antes_partida_trat,nacionalidade_trat
0,DFW,20160712092585830,brasil,GRU,26/06/2021 19:30:00,estadosunidos,Z963,American Airlines,DFW,Dallas-Fort Worth,estadosunidos,GRU,1.0,27062021,1.0,0,014.573.158-82,NaN,BRA,F,brasil,2027-01-22,1961-06-23,10.0,0.0,2.0,0.0,0.0,2.0,"R$ 47.165,87","R$ 62.658,74","R$ 0,00",[2940691],[1],[BAIXADA],[],"[214866808, FS370619, FF505476, FF505476, 2016...","[ESTADOS UNIDOS, BRASIL, ESTADOS UNIDOS, BRASIL]",[ESTADOS UNIDOS],[4],[ESTADOS UNIDOS],[6],"[AA, AA, AA, AA, AA, AA, AA, AA, AA, AA]","[962, 963, 962, 962, 963, 962, 963, 962, 963, ...","[1, 1, 2, 1, 1, 2, 1, 1, 1, 1]","[GRU, DFW, GRU, GRU, DFW, GRU, DFW, GRU, DFW, ...","[DFW, GRU, DFW, DFW, GRU, DFW, GRU, DFW, GRU, ...",25/06/2021,NaN,2021-06-27,1,1,1.0,27,06,2021,2016071209258583027062021,2016071209258583027062021,1.0,9,0.0,2027-01-22 00:00:00,7,True,8,1.0,1.0,0,1,0,0,0,5.0,60.0,2,"[GRU, DFW, GRU, GRU, DFW, GRU, DFW, GRU, DFW, ...","[DFW, GRU, GRU, DFW, GRU, DFW, GRU, DFW, GRU]",4,1,1,1,"[1, 1, 2, 1, 1, 2, 1, 1, 1, 1]",1.222222,2021-06-25,2021-06-26,1 days,brasil
1,IST,20190711121060050,argentina,GRU,27/06/2021 10:20:00,turquia,Z15,Turkish Airlines,ICN,Seoul,coreiadosul,GRU,0.0,27062021,1.0,0,NaN,NaN,ARG,M,argentina,2024-10-10,1986-09-17,7.0,0.0,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[AAC176912, AR2000000053]",[ARGENTINA],"[ARGENTINA, COREIA (DO SUL), REPUBLICA DA]","[4, 1]","[ARGENTINA, COREIA (DO SUL), REPUBLICA DA]","[2, 2]","[TK, ET, ET, G3, G3, JJ, JJ]","[15, 507, 507, 7680, 7689, 8003, 8009]","[1, 1, 1, 1, 1, 1, 1]","[ICN, EZE, EZE, BSB, EZE, GRU, EZE]","[GRU, ICN, ICN, EZE, BSB, EZE, GRU]",23/06/2021,NaN,2021-06-27,1,1,1.0,27,06,2021,2019071112106005027062021,2019071112106005027062021,1.0,6,0.0,2024-10-10 00:00:00,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,34.0,1,"[ICN, EZE, EZE, BSB, EZE, GRU, EZE]","[EZE, EZE, BSB, EZE, GRU, EZE]",0,2,2,4,"[1, 1, 1, 1, 1, 1, 1]",1.000000,2021-06-23,2021-06-27,4 days,argentina
2,CUN,20210610060120110,brasil,GRU,26/06/2021 17:00:00,mexico,Z8199,TAM Brazilian Airlines,CUN,Canc?n,mexico,NVT,0.0,27062021,2.0,0,078.692.439-06,NaN,BRA,M,brasil,2030-09-24,1998-07-10

In [49]:
cod_paises = cod_paises.rename(columns={'nacionalidade_trat':'nacionalidade_vu_trat'})

df_trat['nacionalidade_vu'] = df_trat['nacionalidade_vu'].str.lower()
df_trat = pd.merge(df_trat,cod_paises,left_on='nacionalidade_vu',right_on='cod', how='left')
df_trat['nacionalidade_vu_trat'] = df_trat['nacionalidade_vu_trat'].fillna('brasil') #NA
df_trat = df_trat.drop(columns=['nacionalidade_vu','cod'])
# df_trat['nacionalidade_vu_trat'].head(4)

 CPF x CPNJ - se tiver CPF e não tiver informação no CNPJ, será considerado como 0.

In [50]:
df_trat = df_trat.drop(columns=['cnpj_atividadeeconomicapreferencial_vu'])

In [51]:
# cnpj_qtd_trat
lista_cpf,lista_cnpj,lista_cnpj2 = list(df_trat['cpf_trat']),list(df_trat['cnpj_qtd_trat']),[]
index = 0
for i in lista_cpf:
    if i == True and np.isnan(lista_cnpj[index]):
        lista_cnpj2.append(0)
    else:
        lista_cnpj2.append(lista_cnpj[index])
    index += 1
df_trat['cnpj_qtd_trat'] = lista_cnpj2

In [52]:
# cnpj_nrestabelecimento_trat
lista_cpf,lista_cnpj,lista_cnpj2 = list(df_trat['cpf_trat']),list(df_trat['cnpj_nrestabelecimento_trat']),[]
index = 0
for i in lista_cpf:
    if i == True and np.isnan(lista_cnpj[index]):
        lista_cnpj2.append(0)
    else:
        lista_cnpj2.append(lista_cnpj[index])
    index += 1
df_trat['cnpj_nrestabelecimento_trat'] = lista_cnpj2

In [53]:
# situacao_cnpj_ativa_trat
lista_cpf,lista_cnpj,lista_cnpj2 = list(df_trat['cpf_trat']),list(df_trat['situacao_cnpj_ativa_trat']),[]
index = 0
for i in lista_cpf:
    if i == True and np.isnan(lista_cnpj[index]):
        lista_cnpj2.append(0)
    else:
        lista_cnpj2.append(lista_cnpj[index])
    index += 1
df_trat['situacao_cnpj_ativa_trat'] = lista_cnpj2

In [54]:
# situacao_cnpj_baixada_trat
lista_cpf,lista_cnpj,lista_cnpj2 = list(df_trat['cpf_trat']),list(df_trat['situacao_cnpj_baixada_trat']),[]
index = 0
for i in lista_cpf:
    if i == True and np.isnan(lista_cnpj[index]):
        lista_cnpj2.append(0)
    else:
        lista_cnpj2.append(lista_cnpj[index])
    index += 1
df_trat['situacao_cnpj_baixada_trat'] = lista_cnpj2

In [55]:
# situacao_cnpj_inapta_trat
lista_cpf,lista_cnpj,lista_cnpj2 = list(df_trat['cpf_trat']),list(df_trat['situacao_cnpj_inapta_trat']),[]
index = 0
for i in lista_cpf:
    if i == True and np.isnan(lista_cnpj[index]):
        lista_cnpj2.append(0)
    else:
        lista_cnpj2.append(lista_cnpj[index])
    index += 1
df_trat['situacao_cnpj_inapta_trat'] = lista_cnpj2

In [56]:
# situacao_cnpj_nula_trat
lista_cpf,lista_cnpj,lista_cnpj2 = list(df_trat['cpf_trat']),list(df_trat['situacao_cnpj_nula_trat']),[]
index = 0
for i in lista_cpf:
    if i == True and np.isnan(lista_cnpj[index]):
        lista_cnpj2.append(0)
    else:
        lista_cnpj2.append(lista_cnpj[index])
    index += 1
df_trat['situacao_cnpj_nula_trat'] = lista_cnpj2

In [57]:
# situacao_cnpj_suspensa_trat
lista_cpf,lista_cnpj,lista_cnpj2 = list(df_trat['cpf_trat']),list(df_trat['situacao_cnpj_suspensa_trat']),[]
index = 0
for i in lista_cpf:
    if i == True and np.isnan(lista_cnpj[index]):
        lista_cnpj2.append(0)
    else:
        lista_cnpj2.append(lista_cnpj[index])
    index += 1
df_trat['situacao_cnpj_suspensa_trat'] = lista_cnpj2

 Renda e Patrimônio

In [58]:
df_trat['dirpf_totalrendimentotributavel_vu'] = df_trat['dirpf_totalrendimentotributavel_vu'].apply(lambda x: str(x).replace('.','')).apply(lambda x: str(x).replace(',','.')).apply(lambda x: str(x).replace('R$','')).astype('float64')
df_trat['dirpf_totalrendimentonaotributavel_vu'] = df_trat['dirpf_totalrendimentonaotributavel_vu'].apply(lambda x: str(x).replace('.','')).apply(lambda x: str(x).replace(',','.')).apply(lambda x: str(x).replace('R$','')).astype('float64')
df_trat['dirpf_totalbensdeclarados_vu'] = df_trat['dirpf_totalbensdeclarados_vu'].apply(lambda x: str(x).replace('.','')).apply(lambda x: str(x).replace(',','.')).apply(lambda x: str(x).replace('R$','')).astype('float64')


 Total do rendimento

In [59]:
df_trat['dirpf_totalrendimento_trat'] = df_trat['dirpf_totalrendimentotributavel_vu'] + df_trat['dirpf_totalrendimentonaotributavel_vu']

Forma de Pagamento da passagem

In [62]:
# forma de pagamento
conditions = [
            (df_trat['forma_pgto_pnr'] == '\ Cartao'),
            (df_trat['forma_pgto_pnr'] == '\ Miscelânea'),
            (df_trat['forma_pgto_pnr'] == '\ Dinheiro \ Cartao'),
            (df_trat['forma_pgto_pnr'] == '\ Dinheiro'),  
            (df_trat['forma_pgto_pnr'] == '\ Cartao \ Miscelânea'),
            (df_trat['forma_pgto_pnr'].isna()), 
            ]
df_trat['forma_pgto_pnr_trat'] = np.select(conditions, [1,2,3,4,5,np.nan], default=1)

In [63]:
df_vu = df_trat['cod_vu']
df_vu = df_vu.drop_duplicates()
df_vu.shape

(23622,)

Duração da viagem

In [64]:
df_duracao = pd.read_csv('DURACAO BuscaVU_PUC.csv',sep=';',encoding="latin-1")
df_duracao.columns= ['tipoVoo','ciaAerea','sequencial','numeroVoo','aeroportoOrigem','aeroportoDestino','dataPartida','dataChegada','qtBagagens','localizadorReserva','qtViajantesReserva','tipoViajante','nomeUtilizado','nomeAeroportoOrigem','nomeAeroportoDestino','aeroportoOrigemViajante','aeroportoOrigemLatitude','aeroportoOrigemLongitude','aeroportoOrigemCidade','aeroportoDestinoViajante','aeroportoDestinoLatitude','aeroportoDestinoLongitude','aeroportoDestinoCidade','nomeAeroportoOrigemViajante','nomeAeroportoDestinoViajante','dataNascimento','documentos','apvoNrSq','apvaNrSq','apvjNrSq','assento','duracaoDias','apvjQtPesoBagKg','viagemAnterior','quantidadePesoBagagem','qtdBagagemPesoSaida','qtdBagagemPesoChegada','dataMovimentoNoBrasilFormatada','dataMovimentoNoBrasil','localizadorReservaComQtdViajantes']

C:\Users\06617699416.INTRARFB\Anaconda3\envs\new2Pycaret\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [65]:
def transformar_em_data(dt):
    dia, mes, ano = int(dt[0:2]), int(dt[3:5]), int(dt[6:10])
    data = date(ano, mes, dia)
    return data
    
data_chegada_lista, data_chegada_lista_2 = list(df_duracao['dataChegada'].astype(str)), []
for i in data_chegada_lista:
    data_chegada_lista_2.append(transformar_em_data(i))

df_duracao['data_da_chegada_do_voo'] = data_chegada_lista_2
data_partida_lista, data_partida_lista_2 = list(df_duracao['dataPartida'].astype(str)), []
for i in data_partida_lista:
    data_partida_lista_2.append(transformar_em_data(i))

df_duracao['data_da_partida_do_voo'] = data_partida_lista_2

In [66]:
sequencial = list(df_duracao['sequencial'])
linhas = len(sequencial)
ls_duracao = []
i = 0
while i < linhas-1:
    if df_duracao['sequencial'][i] == df_duracao['sequencial'][i+1]:
        if df_duracao['tipoVoo'][i] == 'E' and df_duracao['tipoVoo'][i+1] == "S":
            ls_duracao.append(str(df_duracao['data_da_chegada_do_voo'][i] - df_duracao['data_da_partida_do_voo'][i+1])) 
        elif df_duracao['tipoVoo'][i] == 'E' and df_duracao['tipoVoo'][i+1] == "E": 
            ls_duracao.append(np.nan) 
        else:
            ls_duracao.append(np.nan) 

    elif df_duracao['sequencial'][i] != df_duracao['sequencial'][i+1]:
        if df_duracao['tipoVoo'][i] == 'E':
            ls_duracao.append(str(-1000))
        else:
            ls_duracao.append(np.nan)

    else:
        ls_duracao.append(np.nan)

    i+=1
ls_duracao.append(np.nan)

df_duracao['duracao_viagem_trat'] = ls_duracao

In [67]:
df_duracao2 = df_duracao.copy()

In [68]:
df_duracao = df_duracao.dropna(axis='rows')

---------------------------------
Merge df_trat + df_duracao

###### Criando código para o merge

In [69]:
def transformar_em_data_chegada_merge(dt):
    dia, mes, ano = str(dt[0:2]), str(dt[3:5]), str(dt[6:10])
    data = dia + mes + ano
    return data

In [70]:
df_duracao['sequencial'] = df_duracao['sequencial'].astype(str)
df_duracao['duracao_viagem_trat'] = df_duracao['duracao_viagem_trat'].astype(str)

In [71]:
lista_data, lista_data_merge = list(df_duracao['dataChegada']), []
for i in lista_data:
    lista_data_merge.append(transformar_em_data_chegada_merge(i)) # transformando a data da chegada
df_duracao['dataChegadaMerge'] = lista_data_merge
df_duracao['vu chegada'] = df_duracao['sequencial'].astype(str) + df_duracao['dataChegadaMerge']
# df_duracao['vu chegada']

In [72]:
df_duracao = df_duracao[['vu chegada','duracao_viagem_trat']]
# df_duracao.head(110)

Merge

In [73]:
df_trat = pd.merge(df_trat,df_duracao, left_on='vu chegada',right_on='vu chegada', how='left')
df_trat.shape

(25786, 89)

In [74]:
df_trat['duracao_viagem_trat'] = df_trat['duracao_viagem_trat'].str.replace(' days, 0:00:00','').str.replace(' day, 0:00:00','').str.replace('0:00:00','0')
df_trat['duracao_viagem_trat'] = df_trat['duracao_viagem_trat'].astype(float)

In [75]:
df_trat2 = df_trat.copy()
df_trat2 = df_trat2.drop(columns=['cod_vu','data_de_partida_do_voo','data_da_chegada_do_voo2',
                                'cpf_vu','cpfresponsavel_vu','documento_validade_vu','datanascimento_vu',
                                'qtviagens_vu','fiscalizacoes_qt_vu','declaracoes_qt_vu',
                                'cnpj_cod_vu','cnpj_nrestabelecimento_vu',
                                'cnpj_situacao_vu',
                                'doc_numdocumento_hist_vu','doc_paisemissor_hist_vu','origem_lista_vu',
                                'origem_qt_vu','destino_lista_vu','destino_qt_vu','ciaaereahist_vu','numerovoohist_vu','qtviajantesreservahist_vu',
                                'aeroportoorigemviajantehist_vu','aeroportodestinoviajantehist_vu',
                                'data_reserva_pnr','data_reserva_pnr_trat','datanascimento_vu','data_da_chegada_do_voo','viagens_periodo_do_vu_trat','qtd_fisc_descendente_trat',
                                'data_merge_dia','data_merge_mes','data_merge_ano','data_merge','vu chegada','num_viagens_api_subtrair','num_viagens_api_subtrair',
                                'data_de_partida_do_voo_trat', 'origem_ls_trat','origem_ls_trat_2',
                                'qtviajantesreservahist_vu2',
                                'doc_ano_validade_trat','cidade_do_embarque_do_viajante','forma_pgto_pnr' ])
                               

In [76]:
df_trat2.columns

Index(['aeroporto_de_origem_do_voo', 'país_emissor_do_documento_principal',
       'codigo_iata_do_destino_do_voo', 'pais_de_origem_do_voo',
       'numero_do_voo', 'companhia_aerea',
       'codigo_iata_do_local_de_embarque_do_passageiro',
       'pais_de_embarque_do_viajante',
       'codigo_iata_do_local_de_desembarque_do_viajante',
       'quantidade_de_bagagens', 'qtd_pax_reserva', 'resultado_reserva',
       'genero_vu', 'documento_emissor_vu', 'monitoramentos_qt_vu',
       'infosinteresse_qt_vu', 'dirpf_tipocontribuinte_vu',
       'dirpf_totalrendimentotributavel_vu',
       'dirpf_totalrendimentonaotributavel_vu', 'dirpf_totalbensdeclarados_vu',
       'fiscalizacoes_qt_vu_trat', 'qtviagens_vu_trat',
       'declaracoes_qt_vu_trat', 'dia_semana_chegada_trat', 'cpf_trat',
       'cpf_regiao_fiscal_trat', 'cnpj_qtd_trat',
       'cnpj_nrestabelecimento_trat', 'situacao_cnpj_ativa_trat',
       'situacao_cnpj_baixada_trat', 'situacao_cnpj_inapta_trat',
       'situacao_cnpj_nula

In [78]:
# df_trat2.to_csv('análise_dados_parte1.csv',sep=';')

-----------------
## Análise prévia 

In [79]:
df_analise = df_trat2.copy()

Dados faltantes

In [80]:
df_nan = pd.DataFrame(df_analise.isna().sum())
df_nan.loc[df_nan[0]>0]

,0
dirpf_tipocontribuinte_vu,7905
dirpf_totalrendimentotributavel_vu,7905
dirpf_totalrendimentonaotributavel_vu,7905
dirpf_totalbensdeclarados_vu,7905
cpf_trat,7904
cpf_regiao_fiscal_trat,7904
cnpj_qtd_trat,7904
cnpj_nrestabelecimento_trat,7904
situacao_cnpj_ativa_trat,7904
situacao_cnpj_baixada_trat,7904


Cardinalidade 

In [87]:
features_categoricas = ['aeroporto_de_origem_do_voo', 'país_emissor_do_documento_principal', 'pais_de_origem_do_voo', 'numero_do_voo', 'companhia_aerea', 'codigo_iata_do_local_de_embarque_do_passageiro', 
                        'pais_de_embarque_do_viajante', 'codigo_iata_do_local_de_desembarque_do_viajante', 'documento_emissor_vu', 'nacionalidade_trat', 'nacionalidade_vu_trat']


In [96]:
df_unique = pd.DataFrame(df_analise[features_categoricas].nunique())
df_unique.loc[df_unique[0]>10]

,0
aeroporto_de_origem_do_voo,36
país_emissor_do_documento_principal,122
pais_de_origem_do_voo,26
numero_do_voo,90
companhia_aerea,23
codigo_iata_do_local_de_embarque_do_passageiro,489
pais_de_embarque_do_viajante,133
codigo_iata_do_local_de_desembarque_do_viajante,62
documento_emissor_vu,121
nacionalidade_trat,128
